In [ ]:
rsync -avzhe ssh \
/media/concha-eloko/Linux/PPT_clean/TropiGATv2.final_df.tsv \
conchae@garnatxa.srv.cpd:/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023/train_nn 



In [1]:
from torch_geometric.data import HeteroData, DataLoader
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, to_hetero , SAGEConv, GATv2Conv, HeteroConv
from torch_geometric.utils import negative_sampling
from torch_geometric.loader import LinkNeighborLoader

import torch
from torch import nn 
import torch.nn.functional as F
import torch.optim as optim

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder , label_binarize , OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score ,matthews_corrcoef

import os 
import pandas as pd
import numpy as np
from tqdm import tqdm
from itertools import product
import random
from collections import Counter
import warnings
warnings.filterwarnings("ignore") 

In [4]:
import TropiGAT_graph
import TropiGAT_models

In [5]:
# *****************************************************************************
# Load the Dataframes :
path_work = "/media/concha-eloko/Linux/PPT_clean"
#path_work = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/ficheros_28032023"

    # Open the DF
DF_info = pd.read_csv(f"{path_work}/TropiGATv2.final_df.tsv", sep = "\t" ,  header = 0)
DF_info_lvl_0 = DF_info[~DF_info["KL_type_LCA"].str.contains("\\|")]
DF_info_lvl_0 = DF_info_lvl_0.drop_duplicates(subset = ["Infected_ancestor","index","prophage_id"] , keep = "first").reset_index(drop=True)



In [7]:
%%time

graph_baseline , dico_prophage_kltype_associated = TropiGAT_graph.build_graph_baseline(DF_info_lvl_0)
#graph_KL64 = TropiGAT_graph.build_graph_masking(graph_baseline , 
#                                 dico_prophage_kltype_associated, 
#                                 DF_info_lvl_0, 
#                                 "KL64",
#                                 2, 0.8, 0.1,0.1)

8815it [00:18, 475.99it/s]

CPU times: user 39.1 s, sys: 739 ms, total: 39.8 s
Wall time: 40.6 s


In [8]:
%%time
graph_dico = {kltype : TropiGAT_graph.build_graph_masking(graph_baseline , dico_prophage_kltype_associated, DF_info_lvl_0, kltype, 2, 0.8, 0.1, 0.1) for kltype in ["KL64", "KL37"]}




CPU times: user 411 ms, sys: 17.1 ms, total: 428 ms
Wall time: 673 ms


In [12]:
graph_KL37 = graph_dico["KL37"]

In [9]:
graph_KL64 = graph_dico["KL64"]

In [10]:
Counter(graph_KL64["B1"].train_mask.numpy())

Counter({False: 6690, True: 2125})

In [13]:
Counter(graph_KL37["B1"].train_mask.numpy())

Counter({False: 8799, True: 16})

In [ ]:
graph_KL37["B1"].train_mask

In [ ]:
for index,value in enumerate(graph_KL37["B1"].y) : 
    if value != 0 :
        print(value,index)

# The architecture of the model : 

***
> Original : 

In [ ]:
# The model : TropiGAT
class TropiGAT_small_module(torch.nn.Module):
    def __init__(self,hidden_channels, heads, edge_type = ("B2", "expressed", "B1") ,dropout = 0.2, conv = GATv2Conv):
        super().__init__()
        # GATv2 module :
        self.conv = conv((-1,-1), hidden_channels, add_self_loops = False, heads = heads, dropout = dropout, shared_weights = True)
        self.hetero_conv = HeteroConv({edge_type: self.conv})
        # FNN layers : 
        self.linear_layers = nn.Sequential(nn.Linear(heads*hidden_channels, 1280),
                                           nn.BatchNorm1d(1280),
                                           nn.LeakyReLU(),
                                           torch.nn.Dropout(dropout),
                                           nn.Linear(1280, 480),
                                           nn.BatchNorm1d(480),
                                           nn.LeakyReLU(),
                                           torch.nn.Dropout(dropout),
                                           nn.Linear(480 , 1))
        
    def forward(self, graph_data):
        x_B1_dict = self.hetero_conv(graph_data.x_dict, graph_data.edge_index_dict)
        x = self.linear_layers(x_B1_dict["B1"])
        return x.view(-1)

class TropiGAT_big_module(torch.nn.Module):
    def __init__(self,hidden_channels, heads, edge_type = ("B2", "expressed", "B1") ,dropout = 0.2, conv = GATv2Conv):
        super().__init__()
        # GATv2 module :
        self.conv = conv((-1,-1), hidden_channels, add_self_loops = False, heads = heads, dropout = dropout, shared_weights = True)
        self.hetero_conv = HeteroConv({edge_type: self.conv})
        # FNN layers : 
        self.linear_layers = nn.Sequential(nn.Linear(heads*hidden_channels, 1280),
                                           nn.BatchNorm1d(1280),
                                           nn.LeakyReLU(),
                                           torch.nn.Dropout(dropout),
                                           nn.Linear(1280, 720),
                                           nn.BatchNorm1d(720),
                                           nn.LeakyReLU(),
                                           torch.nn.Dropout(dropout),
                                           nn.Linear(720 , 240),
                                           nn.BatchNorm1d(240),
                                           nn.LeakyReLU(),
                                           torch.nn.Dropout(dropout),
                                           nn.Linear(240, 1)
                                          )
        
    def forward(self, graph_data):
        x = self.hetero_conv(graph_data.x_dict, graph_data.edge_index_dict)
        x = self.linear_layers(x)
        
        return x.view(-1)

In [ ]:
parameters_model = {"hidden_channels" : 1000,
                    "lr" : 0.0001,
                    "heads" : 1,
                    "dropout" : 0.1,
                   }

TropiGATv2_eg = TropiGAT_models.TropiGAT_small_module(hidden_channels = 1280 , heads = 1)
TropiGATv2_eg.eval()

In [ ]:
with torch.no_grad():
    out = TropiGATv2_eg(graph_KL64)

In [ ]:
len(out)

In [ ]:
Counter(graph_KL64["B1"].train_mask.numpy())

In [ ]:
graph_KL37["B1"].train_mask

In [ ]:
out.argmax(dim=0)
out[graph_KL37["B1"].train_mask]
graph_KL37["B1"].train_mask

In [ ]:
graph_KL37["B1"].y

***
# Training the model 

In [ ]:
def train(model, graph, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    out_train = model(graph)
    loss = criterion(out_train[graph["B1"].train_mask], graph["B1"].y[graph["B1"].train_mask].float())
    loss.backward()
    optimizer.step()
    return loss

@torch.no_grad()
def evaluate(model, graph,criterion, mask):
    model.eval()
    out_eval = model(graph)
    #logging.info(f"Out eval : {out_eval}")
    pred = out_eval[mask]
    pred = torch.sigmoid(out_eval).round()
    #pred = pred.round()
    labels = graph["B1"].y[mask]
    val_loss = criterion(out_eval[mask], graph["B1"].y[mask].float())
    logging.info(f"The predictions : {pred} and length {len(pred)}")
    logging.info(f"The labels after masking : {labels} and the length {len(labels)}. Here are the original labels : {graph['B1'].y} with length {len(graph['B1'].y)} and finally {Counter(graph['B1'].y.numpy())}\nHere are the masked labels again : {graph['B1'].y[mask]}\nHere is the mask used : {mask} with {Counter(mask.numpy())}")
    # Calculate the metrics
    f1 = f1_score(labels, pred, average='binary')
    precision = precision_score(labels, pred, average='binary')
    recall = recall_score(labels, pred, average='binary')
    mcc = matthews_corrcoef(labels, pred)
    accuracy = accuracy_score(labels, pred)
    auc = roc_auc_score(labels, out_eval[mask])
    return val_loss.item(), (f1, precision, recall, mcc, accuracy, auc)


In [ ]:
def train_graph() :
    KL_type = "KL64"
    graph_data_kltype = graph_dico[KL_type]
    #logging.info(f"Let's start the work with {conv}\t{hidden_channels}\t{dropout}\t{lr}\t{heads}")
    model = TropiGAT_models.TropiGAT_big_module(hidden_channels = 1280, heads = 1)
    model(graph_data_kltype)
    optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001 , weight_decay= 0.000001)
    scheduler = ReduceLROnPlateau(optimizer, 'min')
    criterion = torch.nn.BCEWithLogitsLoss()
    for epoch in range(500):
        train_loss = TropiGAT_models.train(model, graph_data_kltype, optimizer,criterion)
        logging.info(train_loss)
        if epoch % 25 == 0:
            # Get all metrics
            test_loss, metrics = TropiGAT_models.evaluate(model, graph_data_kltype,criterion, graph_data_kltype["B1"].test_mask)
            info_training_concise = f'Epoch: {epoch}\tTrain Loss: {train_loss}\tTest Loss: {test_loss}\tMCC: {metrics[3]}\tAUC: {metrics[5]}'
            info_training = f'Epoch: {epoch}, Train Loss: {train_loss}, Test Loss: {test_loss},F1 Score: {metrics[0]}, Precision: {metrics[1]}, Recall: {metrics[2]}, MCC: {metrics[3]},Accuracy: {metrics[4]}, AUC: {metrics[5]}'
            logging.info(info_training_concise)
            print(info_training)
            scheduler.step(test_loss)
    # Save the model
    torch.save(model.state_dict(), f"{path_work}/{kltype}.TropiGATv2.2509.pt")
    # The final eval :
    print("Final evaluation ...")
    test_loss, metrics = TropiGAT_models.evaluate(model, graph_data_kltype, criterion,graph_data_kltype["B1"].eval_mask)
    info_eval = f'Epoch: {epoch}, F1 Score: {metrics[0]}, Precision: {metrics[1]}, Recall: {metrics[2]}, MCC: {metrics[3]},Accuracy: {metrics[4]}, AUC: {metrics[5]}'
    print(info_eval)
    logging.info(f"Final evaluation ...\n{info_eval}")


In [ ]:
%%time
graph_dico = {kltype : TropiGAT_graph.build_graph_masking(graph_baseline , dico_prophage_kltype_associated,DF_info_lvl_0, kltype,2, 0.8, 0.1,0.1) 
             for kltype in DF_info_lvl_0["KL_type_LCA"].unique()}

In [ ]:
Counter(graph_KL64["B1"].test_mask.numpy())